In [1]:
#Import all necessary libraries 
import pandas as pd
import numpy as np
from matplotlib import pyplot
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")
import requests
import pickle

# Solar Plant

From the ML models, in order to predict the Power Generated in MW Output for a Solar Plant the following inputs are needed:
1. Temp Hi	
2. Temp Low	
3. Solar	
4. Cloud Cover Percentage	
5. Rainfall in mm	
From the 7Timer API I will use both Civil Light and Meteo because Civil Light has Temp Hi, Temp Low, Solar and Rainfall in mm data and Meteo has Cloud Cover data.

In [2]:
# Get the data from the weather API: 7timer.info
url = 'http://www.7timer.info/bin/api.pl?lon=141.988&lat=-20.219&product=civillight&output=json'
Solar = requests.get(url).json()
print (Solar)

{'product': 'civillight', 'init': '2021040700', 'dataseries': [{'date': 20210407, 'weather': 'cloudy', 'temp2m': {'max': 36, 'min': 31}, 'wind10m_max': 3}, {'date': 20210408, 'weather': 'cloudy', 'temp2m': {'max': 35, 'min': 26}, 'wind10m_max': 3}, {'date': 20210409, 'weather': 'clear', 'temp2m': {'max': 36, 'min': 26}, 'wind10m_max': 3}, {'date': 20210410, 'weather': 'clear', 'temp2m': {'max': 37, 'min': 26}, 'wind10m_max': 3}, {'date': 20210411, 'weather': 'clear', 'temp2m': {'max': 36, 'min': 25}, 'wind10m_max': 3}, {'date': 20210412, 'weather': 'clear', 'temp2m': {'max': 33, 'min': 23}, 'wind10m_max': 4}, {'date': 20210413, 'weather': 'clear', 'temp2m': {'max': 33, 'min': 23}, 'wind10m_max': 3}]}


In [3]:
# Get the data from the weather API: 7timer.info
url1 = 'http://www.7timer.info/bin/api.pl?lon=141.988&lat=-20.219&product=meteo&output=json'
Solar_Plant = requests.get(url1).json()
print (Solar_Plant)

{'product': 'meteo', 'init': '2021040700', 'dataseries': [{'timepoint': 3, 'cloudcover': 1, 'highcloud': -9999, 'midcloud': -9999, 'lowcloud': -9999, 'rh_profile': [{'layer': '950mb', 'rh': 3}, {'layer': '900mb', 'rh': 5}, {'layer': '850mb', 'rh': 7}, {'layer': '800mb', 'rh': 10}, {'layer': '750mb', 'rh': 14}, {'layer': '700mb', 'rh': 14}, {'layer': '650mb', 'rh': 3}, {'layer': '600mb', 'rh': -3}, {'layer': '550mb', 'rh': -4}, {'layer': '500mb', 'rh': -4}, {'layer': '450mb', 'rh': -4}, {'layer': '400mb', 'rh': -4}, {'layer': '350mb', 'rh': -4}, {'layer': '300mb', 'rh': -4}, {'layer': '250mb', 'rh': -4}, {'layer': '200mb', 'rh': -2}], 'wind_profile': [{'layer': '950mb', 'direction': 170, 'speed': 3}, {'layer': '900mb', 'direction': 170, 'speed': 3}, {'layer': '850mb', 'direction': 175, 'speed': 3}, {'layer': '800mb', 'direction': 180, 'speed': 3}, {'layer': '750mb', 'direction': 185, 'speed': 3}, {'layer': '700mb', 'direction': 185, 'speed': 3}, {'layer': '650mb', 'direction': 175, 'spe

In [4]:
#From Civil Light data, cpllecting date, maximum and minimum temperatures. 
data_size = len(Solar["dataseries"])
date = []
Temp_Hi = []
Temp_Low = []
for i in range(data_size):
    sdate = Solar['dataseries'][i]['date']
    smax = Solar['dataseries'][i]['temp2m']['max']
    smin = Solar['dataseries'][i]['temp2m']['min']
    date.append(sdate)
    Temp_Hi.append(smax)
    Temp_Low.append(smin)

print(date, Temp_Hi, Temp_Low)

[20210407, 20210408, 20210409, 20210410, 20210411, 20210412, 20210413] [36, 35, 36, 37, 36, 33, 33] [31, 26, 26, 26, 25, 23, 23]


In [5]:
#From Meteo data, collecting cloud cover
Cloud_Cover_Percentage =[]
for i in range(7):
    cloud = Solar_Plant['dataseries'][i]['cloudcover']
    Cloud_Cover_Percentage.append(cloud)
    
print (Cloud_Cover_Percentage)

[1, 5, 9, 9, 7, 8, 3]


In [6]:
#Creating a data frame  
data = pd.DataFrame([date,Temp_Hi, Temp_Low, Cloud_Cover_Percentage]).transpose()
data

,0,1,2,3
0,20210407,36,31,1
1,20210408,35,26,5
2,20210409,36,26,9
3,20210410,37,26,9
4,20210411,36,25,7
5,20210412,33,23,8
6,20210413,33,23,3


In [7]:
# Need to convert temp to Frahenhei
data['Temp Hi']= (data[1]*9/5) + 32
data['Temp Low']= (data[2]*9/5) + 32
data

,0,1,2,3,Temp Hi,Temp Low
0,20210407,36,31,1,96.8,87.8
1,20210408,35,26,5,95.0,78.8
2,20210409,36,26,9,96.8,78.8
3,20210410,37,26,9,98.6,78.8
4,20210411,36,25,7,96.8,77.0
5,20210412,33,23,8,91.4,73.4
6,20210413,33,23,3,91.4,73.4


In [8]:
data["Cloud Cover Percentage"] = data[3]
data

,0,1,2,3,Temp Hi,Temp Low,Cloud Cover Percentage
0,20210407,36,31,1,96.8,87.8,1
1,20210408,35,26,5,95.0,78.8,5
2,20210409,36,26,9,96.8,78.8,9
3,20210410,37,26,9,98.6,78.8,9
4,20210411,36,25,7,96.8,77.0,7
5,20210412,33,23,8,91.4,73.4,8
6,20210413,33,23,3,91.4,73.4,3


In [9]:
data.drop([0,1,2,3], inplace =True, axis = 1)# Drop the other columns unnamed columns
data

,Temp Hi,Temp Low,Cloud Cover Percentage
0,96.8,87.8,1
1,95.0,78.8,5
2,96.8,78.8,9
3,98.6,78.8,9
4,96.8,77.0,7
5,91.4,73.4,8
6,91.4,73.4,3


In [10]:
solar_model = pickle.load(open('solar_model.sav','rb'))# load the solar model
Solar_plant_power = solar_model.predict(data.values)# predicted values
Solar_plant_power

array([1.53004263, 5.51221749, 9.50244824, 9.5160296 , 7.51794491,
       8.47900591, 3.50819416])

# Wind Plant

In [11]:
#Getting the Wind data from 7Timer Meteo as it contains all the necessary inputs
url2 = ' http://www.7timer.info/bin/api.pl?lon=8.591&lat=53.562&product=meteo&output=json' 
Wind_Plant = requests.get(url2).json()
print (Wind_Plant)

{'product': 'meteo', 'init': '2021040700', 'dataseries': [{'timepoint': 3, 'cloudcover': 7, 'highcloud': -9999, 'midcloud': -9999, 'lowcloud': -9999, 'rh_profile': [{'layer': '950mb', 'rh': 10}, {'layer': '900mb', 'rh': 13}, {'layer': '850mb', 'rh': 13}, {'layer': '800mb', 'rh': 10}, {'layer': '750mb', 'rh': 7}, {'layer': '700mb', 'rh': 8}, {'layer': '650mb', 'rh': 10}, {'layer': '600mb', 'rh': 10}, {'layer': '550mb', 'rh': 1}, {'layer': '500mb', 'rh': 0}, {'layer': '450mb', 'rh': 1}, {'layer': '400mb', 'rh': 4}, {'layer': '350mb', 'rh': 4}, {'layer': '300mb', 'rh': -1}, {'layer': '250mb', 'rh': -3}, {'layer': '200mb', 'rh': -4}], 'wind_profile': [{'layer': '950mb', 'direction': 285, 'speed': 5}, {'layer': '900mb', 'direction': 290, 'speed': 5}, {'layer': '850mb', 'direction': 290, 'speed': 5}, {'layer': '800mb', 'direction': 295, 'speed': 4}, {'layer': '750mb', 'direction': 300, 'speed': 4}, {'layer': '700mb', 'direction': 305, 'speed': 4}, {'layer': '650mb', 'direction': 315, 'speed'

In [12]:
#data_size = len(Wind_Plant["dataseries"])
wind_speed = []
wind_direction = []
for i in range(7):
    wspeed = Wind_Plant['dataseries'][i]['wind_profile'][0]['speed']
    wdirection = Wind_Plant['dataseries'][i]['wind_profile'][0]['direction']
    wind_speed.append(wspeed)
    wind_direction.append(wdirection)
    
print(wind_speed, wind_direction)

[5, 5, 4, 5, 5, 5, 5] [285, 260, 300, 310, 315, 310, 315]


In [13]:
#Converting the speed values by using average values as specified in the documentation info of the api as per interval to suitable values for model
speed_value ={'1': 0.3, '2': 1.83, '3':5.7, '4':9.4, '5':14, '6':20.85, '7':28.55, '8':34.65, '9':39.05, '10':43.8, '11':48.55, '12':53.4, '13':55.9}
wind_speed = []
wind_direction = []
for i in range(7):
    s = Wind_Plant['dataseries'][i]['wind_profile'][0]['speed']
    wspeed = speed_value[str(s)] #Use created dictionary labeled speed_value
    wdirection = Wind_Plant['dataseries'][i]['wind_profile'][0]['direction']
    wind_speed.append(wspeed)
    wind_direction.append(wdirection)
    
print(wind_speed, wind_direction)

[14, 14, 9.4, 14, 14, 14, 14] [285, 260, 300, 310, 315, 310, 315]


In [14]:
#Creating a data frame 
df = pd.DataFrame([wind_speed, wind_direction]).transpose()
print(df)

      0      1
0  14.0  285.0
1  14.0  260.0
2   9.4  300.0
3  14.0  310.0
4  14.0  315.0
5  14.0  310.0
6  14.0  315.0


In [15]:
#Renaming the columns in the data frame and dropping unnamed columns 
df["wind speed"]=df[0]
df["direction"]=df[1]
df.drop([0,1], inplace=True, axis=1)
df

,wind speed,direction
0,14.0,285.0
1,14.0,260.0
2,9.4,300.0
3,14.0,310.0
4,14.0,315.0
5,14.0,310.0
6,14.0,315.0


In [16]:
wind_model = pickle.load(open('wind_model.sav','rb'))# load the wind model
wind_plant_power = wind_model.predict(df.values)# predicted values
wind_plant_power


array([45.48, 45.48, 45.48, 45.48, 45.48, 45.48, 45.48])